# Load ENV File

In [1]:
from pathlib import Path

from src.custom_types import EnvironmentTypes

from dotenv import load_dotenv

dotenv_path = Path("envs/.env")
load_dotenv(dotenv_path=dotenv_path)
print(f"Loaded {dotenv_path}")

Loaded envs\.env


# Create DB_MANAGER

In [2]:
from src.db.db_manager import DBManager
from src.utils import get_db_settings

db_manager = DBManager(
    db_settings=get_db_settings(),
)

ImportError: cannot import name 'MoneyBoxNotFoundError' from 'src.db.exceptions' (F:\dev\python\savings_manager\src\db\exceptions.py)

# EXAMPLES

## Create Moneyboxes

In [ ]:
import uuid

db_manager = DBManager(
    db_settings=get_db_settings(),
)

GENERATE_N_MONEYBOXES = 10
created_moneyboxes = {}

for _ in range(GENERATE_N_MONEYBOXES):
    moneybox_data = {"name": f"Test Monybox [{uuid.uuid4()}]"}
    moneybox = await db_manager.add_moneybox(moneybox_data=moneybox_data)
    created_moneyboxes[moneybox["id"]] = moneybox
    print(f"Created: {moneybox}")

## Delete Moneyboxes

In [ ]:
from pprint import pprint

#pprint(created_moneyboxes)

delete_moneybox_ids = [moneybox_id for moneybox_id, moneybox in created_moneyboxes.items() if moneybox["id"] % 3 == 0]
print(f"Try to delete Moneyboxex with id: {', '.join(map(str, delete_moneybox_ids))}")


for moneybox_id in delete_moneybox_ids:
    try:
        await db_manager.delete_moneybox(moneybox_id=moneybox_id)
    except Exception as ex:
        print(ex)
